# Getting Started with Meridian Ephemeris API

This notebook demonstrates the basic usage of the Meridian Ephemeris API for calculating natal charts and working with astrological data.

## Prerequisites

Install required packages:
```bash
pip install requests pandas matplotlib seaborn jupyter
```

In [ ]:
# Import required libraries
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timezone
import json
from typing import Dict, Any

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

## 1. API Configuration

Configure the API endpoint and create helper functions for making requests.

In [ ]:
# API Configuration
BASE_URL = "http://localhost:8000"  # Change to https://api.meridianephemeris.com for production

class MeridianEphemerisClient:
    """Simple client for Meridian Ephemeris API"""
    
    def __init__(self, base_url: str = BASE_URL):
        self.base_url = base_url.rstrip('/')
        self.session = requests.Session()
        self.session.headers.update({
            'Content-Type': 'application/json',
            'Accept': 'application/json'
        })
    
    def health_check(self) -> Dict[str, Any]:
        """Check API health"""
        response = self.session.get(f"{self.base_url}/health")
        response.raise_for_status()
        return response.json()
    
    def calculate_natal_chart(self, subject_data: Dict[str, Any], 
                            settings: Dict[str, Any] = None) -> Dict[str, Any]:
        """Calculate a natal chart"""
        payload = {
            "subject": subject_data,
            "settings": settings or {}
        }
        
        response = self.session.post(f"{self.base_url}/ephemeris/natal", json=payload)
        response.raise_for_status()
        return response.json()

# Initialize client
client = MeridianEphemerisClient()

# Test connection
try:
    health = client.health_check()
    print(f"✅ API Health: {health['status']}")
    print(f"📊 Service: {health['service']}")
    print(f"🔢 Version: {health['version']}")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("Make sure the API server is running on http://localhost:8000")

## 2. Your First Chart Calculation

Let's calculate a natal chart for a sample birth data.

In [ ]:
# Sample birth data
sample_subject = {
    "name": "Sample Person",
    "datetime": {"iso_string": "1990-06-15T14:30:00"},
    "latitude": {"decimal": 40.7128},   # New York City
    "longitude": {"decimal": -74.0060},
    "timezone": {"name": "America/New_York"}
}

# Calculate the chart
print("🔮 Calculating natal chart...")
chart_response = client.calculate_natal_chart(sample_subject)

if chart_response["success"]:
    chart_data = chart_response["data"]
    print("✅ Chart calculated successfully!")
    print(f"📅 Birth Date: {chart_data['subject']['datetime']}")
    print(f"📍 Location: {chart_data['subject']['latitude']}, {chart_data['subject']['longitude']}")
    print(f"⏱️ Calculation Time: {chart_response.get('metadata', {}).get('processing_time_ms', 'N/A')}ms")
else:
    print(f"❌ Chart calculation failed: {chart_response.get('message', 'Unknown error')}")
    chart_data = None

## 3. Exploring Planetary Positions

Let's examine the planetary positions in the calculated chart.

In [ ]:
if chart_data:
    planets = chart_data["planets"]
    
    # Create a DataFrame for easier analysis
    planet_data = []
    for planet_name, planet_info in planets.items():
        planet_data.append({
            'Planet': planet_name.title(),
            'Longitude': planet_info['longitude'],
            'Latitude': planet_info.get('latitude', 0),
            'Distance': planet_info.get('distance', 0),
            'Speed': planet_info.get('longitude_speed', 0),
            'Sign': planet_info.get('sign', 'Unknown'),
            'House': planet_info.get('house', 0),
            'Retrograde': planet_info.get('retrograde', False)
        })
    
    df_planets = pd.DataFrame(planet_data)
    print("🪐 Planetary Positions:")
    print(df_planets[['Planet', 'Longitude', 'Sign', 'House', 'Retrograde']].to_string(index=False))

## 4. Visualizing Planetary Distribution

Create visualizations to better understand the chart data.

In [ ]:
if chart_data:
    # Create zodiac wheel visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Planetary Longitudes
    longitudes = df_planets['Longitude'].tolist()
    planet_names = df_planets['Planet'].tolist()
    
    # Convert to radians for polar plot
    angles = [(lng * 3.14159 / 180) for lng in longitudes]
    
    ax1_polar = plt.subplot(121, projection='polar')
    ax1_polar.scatter(angles, [1] * len(angles), c=range(len(angles)), 
                     s=100, cmap='hsv', alpha=0.8)
    
    # Add planet labels
    for i, (angle, name) in enumerate(zip(angles, planet_names)):
        ax1_polar.annotate(name, (angle, 1.1), ha='center', va='center')
    
    ax1_polar.set_ylim(0, 1.5)
    ax1_polar.set_title('Planetary Positions\n(Zodiac Wheel)', pad=20)
    ax1_polar.grid(True)
    
    # Plot 2: House Distribution
    house_counts = df_planets['House'].value_counts().sort_index()
    
    ax2.bar(house_counts.index, house_counts.values, color='skyblue', alpha=0.7)
    ax2.set_xlabel('House')
    ax2.set_ylabel('Number of Planets')
    ax2.set_title('Planetary Distribution by House')
    ax2.set_xticks(range(1, 13))
    ax2.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Element and modality distribution
    print("\n🔥 Sign Distribution:")
    sign_counts = df_planets['Sign'].value_counts()
    print(sign_counts.to_string())

## 5. House System Analysis

Examine the house system and angular positions.

In [ ]:
if chart_data:
    houses = chart_data["houses"]
    
    print(f"🏠 House System: {houses['system'].title()}")
    print("\n📐 Angular Positions:")
    
    angles = houses['angles']
    angle_names = {
        'ascendant': 'Ascendant (AC)',
        'midheaven': 'Midheaven (MC)', 
        'descendant': 'Descendant (DC)',
        'imum_coeli': 'Imum Coeli (IC)'
    }
    
    for angle_key, angle_value in angles.items():
        if angle_key in angle_names:
            print(f"  {angle_names[angle_key]}: {angle_value:.2f}°")
    
    # House cusps
    print("\n🏘️ House Cusps:")
    cusps = houses['cusps']
    for i, cusp in enumerate(cusps, 1):
        print(f"  House {i:2d}: {cusp:7.2f}°")
    
    # Visualize house sizes
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Calculate house sizes
    house_sizes = []
    for i in range(12):
        next_cusp = cusps[(i + 1) % 12]
        current_cusp = cusps[i]
        
        size = (next_cusp - current_cusp) % 360
        if size == 0:
            size = 360
        house_sizes.append(size)
    
    house_numbers = list(range(1, 13))
    bars = ax.bar(house_numbers, house_sizes, color='lightcoral', alpha=0.7)
    
    # Highlight angular houses
    angular_houses = [1, 4, 7, 10]
    for i, bar in enumerate(bars):
        if i + 1 in angular_houses:
            bar.set_color('darkred')
            bar.set_alpha(0.8)
    
    ax.axhline(y=30, color='gray', linestyle='--', alpha=0.5, label='Equal House (30°)')
    ax.set_xlabel('House Number')
    ax.set_ylabel('House Size (degrees)')
    ax.set_title(f'House Sizes - {houses["system"].title()} System')
    ax.set_xticks(house_numbers)
    ax.grid(axis='y', alpha=0.3)
    ax.legend()
    
    plt.tight_layout()
    plt.show()

## 6. Comparing Different House Systems

Let's compare how different house systems affect the chart.

In [ ]:
# Available house systems
house_systems = ['placidus', 'koch', 'equal', 'whole_sign', 'campanus']

# Calculate charts with different house systems
house_system_data = {}

print("🔄 Calculating charts with different house systems...")

for system in house_systems:
    try:
        settings = {"house_system": system}
        response = client.calculate_natal_chart(sample_subject, settings)
        
        if response["success"]:
            house_system_data[system] = response["data"]["houses"]
            print(f"✅ {system.title()} system calculated")
        else:
            print(f"❌ {system.title()} system failed: {response.get('message', 'Unknown error')}")
    except Exception as e:
        print(f"❌ {system.title()} system error: {e}")

# Compare Ascendant positions
if house_system_data:
    print("\n📊 Ascendant Comparison:")
    for system, data in house_system_data.items():
        asc = data['angles']['ascendant']
        print(f"  {system.title():12}: {asc:7.2f}°")
    
    # Plot house cusp comparison
    fig, ax = plt.subplots(figsize=(12, 8))
    
    house_numbers = list(range(1, 13))
    width = 0.15
    x = range(len(house_numbers))
    
    colors = ['red', 'blue', 'green', 'orange', 'purple']
    
    for i, (system, data) in enumerate(house_system_data.items()):
        cusps = data['cusps']
        offset = (i - len(house_system_data)/2 + 0.5) * width
        ax.bar([pos + offset for pos in x], cusps, width, 
               label=system.title(), alpha=0.7, color=colors[i % len(colors)])
    
    ax.set_xlabel('House Number')
    ax.set_ylabel('House Cusp Position (degrees)')
    ax.set_title('House Cusp Comparison Across Different Systems')
    ax.set_xticks(x)
    ax.set_xticklabels(house_numbers)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 7. Working with Different Input Formats

The API supports multiple input formats for coordinates, dates, and timezones.

In [ ]:
# Different coordinate formats
coordinate_examples = {
    "decimal": {
        "latitude": {"decimal": 51.5074},
        "longitude": {"decimal": -0.1278},
        "location": "London, UK"
    },
    "dms_string": {
        "latitude": {"dms_string": "51°30'27\"N"},
        "longitude": {"dms_string": "0°07'40\"W"},
        "location": "London, UK (DMS)"
    },
    "component": {
        "latitude": {
            "degrees": 51,
            "minutes": 30,
            "seconds": 27,
            "direction": "N"
        },
        "longitude": {
            "degrees": 0,
            "minutes": 7,
            "seconds": 40,
            "direction": "W"
        },
        "location": "London, UK (Components)"
    }
}

# Test different formats
print("🌍 Testing different coordinate formats...")

for format_name, coords in coordinate_examples.items():
    test_subject = {
        "name": f"Test - {coords['location']}",
        "datetime": {"iso_string": "2000-01-01T12:00:00"},
        "latitude": coords["latitude"],
        "longitude": coords["longitude"],
        "timezone": {"name": "Europe/London"}
    }
    
    try:
        response = client.calculate_natal_chart(test_subject)
        if response["success"]:
            data = response["data"]
            actual_lat = data["subject"]["latitude"]
            actual_lng = data["subject"]["longitude"]
            print(f"✅ {format_name:10}: {actual_lat:.4f}, {actual_lng:.4f}")
        else:
            print(f"❌ {format_name:10}: {response.get('message', 'Failed')}")
    except Exception as e:
        print(f"❌ {format_name:10}: {e}")

# Different datetime formats
datetime_examples = {
    "iso_string": {"iso_string": "2000-06-21T12:00:00"},
    "julian_day": {"julian_day": 2451716.0},
    "components": {
        "year": 2000,
        "month": 6,
        "day": 21,
        "hour": 12,
        "minute": 0,
        "second": 0
    }
}

print("\n📅 Testing different datetime formats...")

for format_name, dt_format in datetime_examples.items():
    test_subject = {
        "name": f"DateTime Test - {format_name}",
        "datetime": dt_format,
        "latitude": {"decimal": 0.0},
        "longitude": {"decimal": 0.0},
        "timezone": {"name": "UTC"}
    }
    
    try:
        response = client.calculate_natal_chart(test_subject)
        if response["success"]:
            actual_dt = response["data"]["subject"]["datetime"]
            print(f"✅ {format_name:12}: {actual_dt}")
        else:
            print(f"❌ {format_name:12}: {response.get('message', 'Failed')}")
    except Exception as e:
        print(f"❌ {format_name:12}: {e}")

## 8. Error Handling and Validation

Learn how to handle API errors and validation issues.

In [ ]:
# Test various error conditions
error_test_cases = [
    {
        "name": "Invalid Latitude",
        "subject": {
            "name": "Error Test",
            "datetime": {"iso_string": "2000-01-01T12:00:00"},
            "latitude": {"decimal": 95.0},  # Invalid: > 90°
            "longitude": {"decimal": 0.0},
            "timezone": {"name": "UTC"}
        }
    },
    {
        "name": "Invalid Date",
        "subject": {
            "name": "Error Test",
            "datetime": {"iso_string": "invalid-date"},
            "latitude": {"decimal": 0.0},
            "longitude": {"decimal": 0.0},
            "timezone": {"name": "UTC"}
        }
    },
    {
        "name": "Missing Fields",
        "subject": {
            "name": "Error Test"
            # Missing required fields
        }
    },
    {
        "name": "Invalid Timezone",
        "subject": {
            "name": "Error Test",
            "datetime": {"iso_string": "2000-01-01T12:00:00"},
            "latitude": {"decimal": 0.0},
            "longitude": {"decimal": 0.0},
            "timezone": {"name": "Invalid/Timezone"}
        }
    }
]

print("🚨 Testing error handling...")

for test_case in error_test_cases:
    print(f"\n🧪 Test Case: {test_case['name']}")
    
    try:
        response = client.calculate_natal_chart(test_case["subject"])
        
        if not response["success"]:
            print(f"  ❌ Error Type: {response.get('error', 'unknown')}")
            print(f"  📝 Message: {response.get('message', 'No message')}")
            
            # Show validation details if available
            if "details" in response and "errors" in response["details"]:
                print("  🔍 Validation Errors:")
                for error in response["details"]["errors"][:3]:  # Show first 3 errors
                    field = " -> ".join(str(loc) for loc in error.get("loc", []))
                    message = error.get("msg", "")
                    print(f"    • {field}: {message}")
        else:
            print(f"  ✅ Unexpected success (this might indicate an issue)")
            
    except requests.exceptions.HTTPError as e:
        print(f"  ❌ HTTP Error: {e}")
        try:
            error_data = e.response.json()
            print(f"  📝 API Error: {error_data.get('message', 'No details')}")
        except:
            pass
    except Exception as e:
        print(f"  ❌ Unexpected Error: {e}")

print("\n✅ Error handling tests completed")

## 9. Performance Analysis

Analyze API performance and caching behavior.

In [ ]:
import time

def measure_calculation_time(subject_data, iterations=3):
    """Measure calculation time over multiple iterations"""
    times = []
    
    for i in range(iterations):
        start_time = time.time()
        response = client.calculate_natal_chart(subject_data)
        end_time = time.time()
        
        if response["success"]:
            duration = (end_time - start_time) * 1000  # Convert to milliseconds
            times.append(duration)
            print(f"  Iteration {i+1}: {duration:.1f}ms")
        else:
            print(f"  Iteration {i+1}: Failed")
    
    return times

# Test caching behavior
print("⏱️ Testing API performance and caching...")

test_subject_perf = {
    "name": "Performance Test",
    "datetime": {"iso_string": "1985-12-25T09:15:00"},
    "latitude": {"decimal": 34.0522},  # Los Angeles
    "longitude": {"decimal": -118.2437},
    "timezone": {"name": "America/Los_Angeles"}
}

print("\n🔄 First calculation (cache miss):")
first_times = measure_calculation_time(test_subject_perf, 3)

print("\n🚀 Subsequent calculations (potential cache hits):")
second_times = measure_calculation_time(test_subject_perf, 3)

if first_times and second_times:
    avg_first = sum(first_times) / len(first_times)
    avg_second = sum(second_times) / len(second_times)
    
    print(f"\n📊 Performance Summary:")
    print(f"  First calculation average: {avg_first:.1f}ms")
    print(f"  Cached calculation average: {avg_second:.1f}ms")
    
    if avg_first > avg_second:
        speedup = avg_first / avg_second
        print(f"  🚀 Cache speedup: {speedup:.1f}x faster")
    else:
        print(f"  📈 No significant caching benefit detected")
    
    # Plot performance comparison
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x = range(1, 4)
    ax.plot(x, first_times, 'o-', label='First Run', linewidth=2, markersize=8)
    ax.plot(x, second_times, 's-', label='Cached Run', linewidth=2, markersize=8)
    
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Response Time (ms)')
    ax.set_title('API Performance: Cache vs Non-Cache')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 10. Next Steps

Congratulations! You've successfully learned the basics of working with the Meridian Ephemeris API. Here are some next steps to explore:

### Advanced Features
- **Aspect Calculations**: Calculate planetary aspects and orbs
- **Fixed Stars**: Include fixed star positions in your charts
- **Transit Calculations**: Calculate planetary transits
- **Progressive Charts**: Secondary progressions and solar returns

### Integration Examples
- **Batch Processing**: Process multiple charts efficiently
- **Database Integration**: Store and retrieve chart data
- **Web Applications**: Build astrological web services
- **Data Analysis**: Analyze patterns across large datasets

### Resources
- **API Documentation**: Complete endpoint reference
- **Client SDKs**: Type-safe libraries for Python, TypeScript, Go
- **Example Projects**: Real-world usage examples
- **Community**: Join discussions and get support

### Support
- 📧 Email: support@meridianephemeris.com
- 🐛 Issues: GitHub repository
- 💬 Community: Discord server
- 📚 Documentation: https://docs.meridianephemeris.com

In [ ]:
print("🎉 Tutorial completed successfully!")
print("\n🚀 You're now ready to build amazing astrological applications!")
print("\n📚 Check out the other notebooks in this series:")
print("  • 02-advanced-calculations.ipynb - Advanced chart techniques")
print("  • 03-batch-processing.ipynb - Processing multiple charts")
print("  • 04-data-analysis.ipynb - Astrological data analysis")
print("  • 05-web-integration.ipynb - Building web applications")